In [0]:
df = spark.read.load("/Volumes/dai/phase2/gold/gold_predctions")
display(df.head(10))

CustomerID,total_spent,total_transactions,total_quantity,last_purchase_date,is_high_valued,prediction,probability
15937,145.35,7,73,2011-10-06,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}"
17669,3406.7200000000003,68,1916,2011-10-28,1,1.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.0"",""1.0""]}"
13258,698.5799999999999,38,287,2011-11-28,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}"
16810,275.64,16,112,2011-05-31,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}"
14273,559.9700000000001,29,264,2011-10-19,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}"
18205,517.36,32,192,2011-11-03,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}"
13494,316.26,18,162,2011-02-01,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}"
14142,311.81000000000006,22,313,2010-12-01,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}"
16323,207.50000000000003,1,50,2011-05-27,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}"
14862,841.2000000000002,48,772,2011-11-14,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}"


#### Extract  `probabililty` of class 1:

In [0]:
from pyspark.sql.functions import udf
from pyspark.ml.linalg import VectorUDT
from pyspark.sql.types import DoubleType

def get_high_value_prob(prob_vector):
    return float(prob_vector[1])

get_high_value_prob_udf = udf(
    get_high_value_prob,
    DoubleType()
)

scored_df = df.withColumn(
    "high_value_prob",
    get_high_value_prob_udf(col("probability"))
)

display(scored_df.head(10))

CustomerID,total_spent,total_transactions,total_quantity,last_purchase_date,is_high_valued,prediction,probability,high_value_prob
15937,145.35,7,73,2011-10-06,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}",5.72094908978694E-5
17669,3406.7200000000003,68,1916,2011-10-28,1,1.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.0"",""1.0""]}",1.0
13258,698.5799999999999,38,287,2011-11-28,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}",5.72094908978694E-5
16810,275.64,16,112,2011-05-31,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}",5.72094908978694E-5
14273,559.9700000000001,29,264,2011-10-19,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}",5.72094908978694E-5
18205,517.36,32,192,2011-11-03,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}",5.72094908978694E-5
13494,316.26,18,162,2011-02-01,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}",5.72094908978694E-5
14142,311.81000000000006,22,313,2010-12-01,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}",5.72094908978694E-5
16323,207.50000000000003,1,50,2011-05-27,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}",5.72094908978694E-5
14862,841.2000000000002,48,772,2011-11-14,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}",5.72094908978694E-5


#### Filter Top Buyers

In [0]:
top_buyers = scored_df.filter(
    (col("prediction") == 1) &
    (col("high_value_prob") > 0.65)
)

In [0]:
top_buyers = top_buyers.select("CustomerID","total_spent","total_transactions","total_quantity","last_purchase_date","high_value_prob").orderBy(
    col("high_value_prob").desc()
)

In [0]:
top_buyers.count(),len(top_buyers.columns)

(897, 6)

In [0]:
spark.read.load("/Volumes/dai/phase2/silver/final_data").count()

4373

---

### Observation:

> Out of `4373 Customer`, Only 897 Customers are `Top Buyers`

---

In [0]:
display(top_buyers.head(10))

CustomerID,total_spent,total_transactions,total_quantity,last_purchase_date,high_value_prob
17669,3406.7200000000003,68,1916,2011-10-28,1.0
12370,3545.689999999998,167,2353,2011-10-19,1.0
17082,2003.8500000000008,81,1049,2011-11-06,1.0
17596,2827.929999999999,270,4400,2011-11-16,1.0
17949,52750.84000000001,79,27571,2011-12-08,1.0
15615,10360.03999999999,429,5493,2011-11-29,1.0
15311,59419.34000000011,2491,37720,2011-12-09,1.0
14680,26932.339999999997,320,12474,2011-11-14,1.0
17365,4801.559999999999,468,4777,2011-11-27,1.0
14713,2664.2599999999998,341,1613,2011-11-30,1.0


In [0]:
top_buyers.write.format("delta").mode("overwrite").save("/Volumes/dai/phase2/gold/top_buyers")